# Introduction to reusable component - Part 1

In [ ]:
import kfp.compiler as compiler
import kfp.components as comp
import kfp.dsl as dsl
import kfp.gcp as gcp

from kfp import Client as KfpClient

import os

In [ ]:
component_store = comp.ComponentStore(
  local_search_paths=['components'])

In [ ]:
write_to_gcs_op = component_store.load_component('write_to_gcs')

In [ ]:
print(comp.func_to_component_text(write_to_gcs_op))

In [ ]:
def print_value(x: str) -> None:
    """Helper funtion to print parameter's value"""
    print(x)

In [ ]:
print_value_op = comp.func_to_container_op(print_value)

In [ ]:
@dsl.pipeline(
    name = 'Write from GCS',
    description = 'Simple pipeline to exeperiment with KFP'
)
def write_to_GCS(
    output_uri_template,
    content = dsl.PipelineParam(
        'content',
        value = 'ciao',
        param_type = dsl.types.String
    )):
    
    writing_task = write_to_gcs_op(
        output_uri_template=output_uri_template, 
        content=content).apply(gcp.use_gcp_secret('user-gcp-sa'))
    
    print_value_task = print_value_op('%s' % writing_task.output)

Compile pipeline to check for errors

In [ ]:
compiler.Compiler().compile(write_to_GCS, write_to_GCS.__name__ + '.pipeline.zip')

If running outside of the cluster with Kubeflow, set `GOOGLE_APPLICATION_CREDENTIALS` for dealing with authorisation. The service account needs to have the role `IAP-secured Web App User`.

In [ ]:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '' # path to the json file of the service account used to log in: it need to have role IAP-secured Web App User
# HOST = '' # url of the cluster e.g. https://demo-kubeflow.endpoints.lf-ml-demo.cloud.goog/pipeline
# CLIENT_ID = '' # The client ID used by Identity-Aware Proxy
# NAMESPACE = '' # user namespace e.g. https://demo-kubeflow.endpoints.lf-ml-demo.cloud.goog/pipeline

In [ ]:
client = KfpClient(
# we are running into the same Kubeflow so we do not need to do anything
#     host=HOST,
#     client_id=CLIENT_ID,
#     namespace=NAMESPACE  
)

Update the ```output_uri_template```, then create a run from the pipeline function. 

In [ ]:
client.create_run_from_pipeline_func(
    write_and_read_from_GCS, 
    {'output_uri_template': 'gs://kfp-primer-workshop/kfp/primer/{{workflow.uid}}/{{pod.name}}/data'},
    experiment_name='01_reusable_components',
    run_name='001')